# Data Challenge 1

## Disclaimer:
    - ** If you want to test the speed of my algorithm, comment out all of the other algos and only run LASSO **, included other algos to show process of assignment
    - Initially I had a model that produced KNN algorithm as the best model with an MSE: 22493.16 and RMSE: 149.97
    - But there was a problem in my feature engineering where when I split my activity period into month and year (to account for seasonal trends in months ie summer and winter), my month and year were string objects causing my MSE/RMSE to look better than it actually was
    - My model is now fixed but produces not as great of an MSE/RMSE, but Professor Brizan says my procedure looks right (at a glance)

# Exploratory Data Analysis
## Goals:
    - Get a feel for the data
    - Remove any necessary data

In [31]:
import pandas as pd

df_train = pd.read_csv("cargo_train.csv")
df_train

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Metric TONS
0,201601,Sun Country Airlines,SY,Sun Country Airlines,SY,Domestic,US,Enplaned,Cargo,Passenger,8.871509
1,200811,United Airlines,UA,United Airlines,UA,Domestic,US,Deplaned,Cargo,Passenger,53.946194
2,200709,Qantas Airways,QF,Qantas Airways,QF,International,Australia / Oceania,Deplaned,Express,Passenger,0.043999
3,201608,Philippine Airlines,PR,Philippine Airlines,PR,International,Asia,Deplaned,Mail,Passenger,19.023530
4,201708,Singapore Airlines,SQ,Singapore Airlines,SQ,International,Asia,Deplaned,Express,Passenger,6.951420
...,...,...,...,...,...,...,...,...,...,...,...
27225,201108,United Airlines - Pre 07/01/2013,UA,United Airlines - Pre 07/01/2013,UA,International,Canada,Enplaned,Express,Passenger,5.687690
27226,200703,Asiana Airlines,OZ,Asiana Airlines,OZ,International,Asia,Deplaned,Mail,Freighter,3.473669
27227,201204,US Airways,US,US Airways,US,Domestic,US,Deplaned,Cargo,Passenger,18.564941
27228,201005,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Cargo,Passenger,9.814090


In [34]:
# look at the makeup of my data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27230 entries, 0 to 27229
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Activity Period              27230 non-null  int64  
 1   Operating Airline            27230 non-null  object 
 2   Operating Airline IATA Code  27199 non-null  object 
 3   Published Airline            27230 non-null  object 
 4   Published Airline IATA Code  27199 non-null  object 
 5   GEO Summary                  27230 non-null  object 
 6   GEO Region                   27230 non-null  object 
 7   Activity Type Code           27230 non-null  object 
 8   Cargo Type Code              27230 non-null  object 
 9   Cargo Aircraft Type          27230 non-null  object 
 10  Cargo Metric TONS            27230 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 2.3+ MB


In [35]:
# Checking for null values
df_train.isna().sum()

Activity Period                 0
Operating Airline               0
Operating Airline IATA Code    31
Published Airline               0
Published Airline IATA Code    31
GEO Summary                     0
GEO Region                      0
Activity Type Code              0
Cargo Type Code                 0
Cargo Aircraft Type             0
Cargo Metric TONS               0
dtype: int64

In [38]:
# drop null values
df_train = df_train.dropna()

In [39]:
# Confirm null values are gone
df_train.isna().sum()

Activity Period                0
Operating Airline              0
Operating Airline IATA Code    0
Published Airline              0
Published Airline IATA Code    0
GEO Summary                    0
GEO Region                     0
Activity Type Code             0
Cargo Type Code                0
Cargo Aircraft Type            0
Cargo Metric TONS              0
dtype: int64

# We need to see association between Operating Airline, Operating Airline IATA Code, Published Airline, and Published Airline IATA Code to see if we can delete columns to have a smaller dataframe (data frame will be too large once I 1 hot encode)

In [40]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(index=[df_train['Operating Airline'], df_train['Published Airline']],
                                columns=[df_train["Operating Airline IATA Code"], df_train["Published Airline IATA Code"]])

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-squared statistic:", chi2)
print("P-value:", p)

Chi-squared statistic: 2393512.0
P-value: 0.0


In [41]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(index=[df_train['Operating Airline IATA Code'], df_train['Published Airline']],
                                columns=[df_train["Operating Airline"], df_train["Published Airline IATA Code"]])

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-squared statistic:", chi2)
print("P-value:", p)

Chi-squared statistic: 2447910.0
P-value: 0.0


# Above
    - The columns have very strong assosciation due to a high chi-squared stat and a p-value of 0.0
    - Columns have strong assosciation is the p-value < 0.05
    - Thus we can use 1/4 of the columns and greatly shrink our dataframe when we need to 1 hot encode many columns later
    - Im going to keep the Operating Airline since it is actually the company flying the plane

In [42]:
cols_to_drop = ['Operating Airline IATA Code', 'Published Airline', 'Published Airline IATA Code']
df_train = df_train.drop(cols_to_drop, axis=1)

In [43]:
df_train

,Activity Period,Operating Airline,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Metric TONS
0,201601,Sun Country Airlines,Domestic,US,Enplaned,Cargo,Passenger,8.871509
1,200811,United Airlines,Domestic,US,Deplaned,Cargo,Passenger,53.946194
2,200709,Qantas Airways,International,Australia / Oceania,Deplaned,Express,Passenger,0.043999
3,201608,Philippine Airlines,International,Asia,Deplaned,Mail,Passenger,19.023530
4,201708,Singapore Airlines,International,Asia,Deplaned,Express,Passenger,6.951420
...,...,...,...,...,...,...,...,...
27225,201108,United Airlines - Pre 07/01/2013,International,Canada,Enplaned,Express,Passenger,5.687690
27226,200703,Asiana Airlines,International,Asia,Deplaned,Mail,Freighter,3.473669
27227,201204,US Airways,Domestic,US,Deplaned,Cargo,Passenger,18.564941
27228,201005,Air Canada,International,Canada,Deplaned,Cargo,Passenger,9.814090


# Above:
    - We removed 3 columns that could essentially be represented by a single column

# Need to 1 hot encode columns 1-6
    - We do this because columns 1-6 are strings and they need to be represented by int values

In [44]:
df_train.nunique()

Activity Period          186
Operating Airline         89
GEO Summary                2
GEO Region                 9
Activity Type Code         2
Cargo Type Code            3
Cargo Aircraft Type        3
Cargo Metric TONS      24872
dtype: int64

In [45]:
# PREP TEST DATA TO BE ONE HOT ENCODED
# same procedure that we did with train data
# preparing test + 1 hot encode
df_test = pd.read_csv("cargo_test.csv")

cols_to_drop = ['Operating Airline IATA Code', 'Published Airline', 'Published Airline IATA Code']
df_test = df_test.drop(cols_to_drop, axis=1)
df_test

,Activity Period,Operating Airline,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type
0,200907,United Airlines,Domestic,US,Deplaned,Mail,Passenger
1,202001,Nippon Cargo Airlines,International,Asia,Deplaned,Cargo,Freighter
2,201107,China Airlines,International,Asia,Enplaned,Cargo,Passenger
3,201809,China Airlines,International,Asia,Deplaned,Cargo,Passenger
4,200811,China Cargo Airlines,International,Asia,Enplaned,Cargo,Freighter
...,...,...,...,...,...,...,...
6802,201603,Air Canada,International,Canada,Enplaned,Mail,Passenger
6803,201910,British Airways,International,Europe,Enplaned,Cargo,Passenger
6804,201503,Air New Zealand,International,Australia / Oceania,Deplaned,Mail,Passenger
6805,201009,World Airways,Domestic,US,Enplaned,Cargo,Freighter


In [46]:
df_train

,Activity Period,Operating Airline,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Metric TONS
0,201601,Sun Country Airlines,Domestic,US,Enplaned,Cargo,Passenger,8.871509
1,200811,United Airlines,Domestic,US,Deplaned,Cargo,Passenger,53.946194
2,200709,Qantas Airways,International,Australia / Oceania,Deplaned,Express,Passenger,0.043999
3,201608,Philippine Airlines,International,Asia,Deplaned,Mail,Passenger,19.023530
4,201708,Singapore Airlines,International,Asia,Deplaned,Express,Passenger,6.951420
...,...,...,...,...,...,...,...,...
27225,201108,United Airlines - Pre 07/01/2013,International,Canada,Enplaned,Express,Passenger,5.687690
27226,200703,Asiana Airlines,International,Asia,Deplaned,Mail,Freighter,3.473669
27227,201204,US Airways,Domestic,US,Deplaned,Cargo,Passenger,18.564941
27228,201005,Air Canada,International,Canada,Deplaned,Cargo,Passenger,9.814090


## Above:
    - training data and testing data have the same columns except training data has the target column (expected)
## Below:
    - We can now 1 hot encode both data sets

In [47]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Create an instance of the OneHotEncoder
encoder_train = OneHotEncoder(handle_unknown='ignore')

columns_to_encode = ['Operating Airline', 'GEO Summary', 'GEO Region', 'Activity Type Code',
                    'Cargo Type Code', 'Cargo Aircraft Type']

# Initialize an empty DataFrame to store the encoded data
encoded_df_train = pd.DataFrame()
encoded_df_test = pd.DataFrame()

# Create a list to store the columns to drop from the original DataFrame
columns_to_drop = []

encoder_y = df_train['Cargo Metric TONS']
encoder_X = df_train.drop(columns=['Cargo Metric TONS'])


# Loop through the columns to encode
for column_name in columns_to_encode:

    if column_name in encoder_X.columns:
        column_to_encode_train = encoder_X[[column_name]]
        column_to_encode_test = df_test[[column_name]]
        
        encoded_data_train = encoder_train.fit_transform(column_to_encode_train)
        encoded_data_test = encoder_train.transform(column_to_encode_test)
        
        
        # Convert to array and create a DataFrame
        encoded_column_df_train = pd.DataFrame(encoded_data_train.toarray())
        encoded_column_df_test = pd.DataFrame(encoded_data_test.toarray())

        
        # Add column names to the encoded DataFrame
        encoded_column_df_train.columns = encoder_train.get_feature_names_out(input_features=[column_name])
        encoded_column_df_test.columns = encoder_train.get_feature_names_out(input_features=[column_name])

        
        # Concatenate the encoded DataFrame with new columns
        encoded_df_train = pd.concat([encoded_df_train, encoded_column_df_train], axis=1)
        encoded_df_test = pd.concat([encoded_df_test, encoded_column_df_test], axis=1)

        
        # Add the column to the list of columns to drop from original dataframe
        columns_to_drop.append(column_name)
        
    else:
        # error checking
        print(f"Column '{column_name}' does not exist in the DataFrame.")

# Drop the original columns that were NOT one-hot encoded
encoder_X = encoder_X.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)


# Concatenate the original DataFrame with the one-hot encoded DataFrame
combined_df = pd.concat([encoder_X, encoded_df_train, encoder_y], axis=1)
combined_df_test = pd.concat([df_test, encoded_df_test], axis=1)

In [48]:
combined_df

,Activity Period,Operating Airline_ABX Air,Operating Airline_ATA Airlines,Operating Airline_Aer Lingus,Operating Airline_Aeromexico,Operating Airline_Air Berlin,Operating Airline_Air Canada,Operating Airline_Air Canada Jazz,Operating Airline_Air Cargo Carriers,Operating Airline_Air China,...,GEO Region_US,Activity Type Code_Deplaned,Activity Type Code_Enplaned,Cargo Type Code_Cargo,Cargo Type Code_Express,Cargo Type Code_Mail,Cargo Aircraft Type_Combi,Cargo Aircraft Type_Freighter,Cargo Aircraft Type_Passenger,Cargo Metric TONS
0,201601.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,8.871509
1,200811.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,53.946194
2,200709.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.043999
3,201608.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,19.023530
4,201708.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,6.951420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22593,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN
23324,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN
23636,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN
23653,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN


In [49]:
combined_df_test

,Activity Period,Operating Airline_ABX Air,Operating Airline_ATA Airlines,Operating Airline_Aer Lingus,Operating Airline_Aeromexico,Operating Airline_Air Berlin,Operating Airline_Air Canada,Operating Airline_Air Canada Jazz,Operating Airline_Air Cargo Carriers,Operating Airline_Air China,...,GEO Region_South America,GEO Region_US,Activity Type Code_Deplaned,Activity Type Code_Enplaned,Cargo Type Code_Cargo,Cargo Type Code_Express,Cargo Type Code_Mail,Cargo Aircraft Type_Combi,Cargo Aircraft Type_Freighter,Cargo Aircraft Type_Passenger
0,200907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,201107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,201809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,200811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,201603,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
6803,201910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
6804,201503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6805,201009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


## Above:
    - Great, the training and test columns look right
    - the training df should have one more col than testing sf
## Below:
    - Need to get rid of null values

In [50]:
combined_df.isna().sum()

Activity Period                   31
Operating Airline_ABX Air         31
Operating Airline_ATA Airlines    31
Operating Airline_Aer Lingus      31
Operating Airline_Aeromexico      31
                                  ..
Cargo Type Code_Mail              31
Cargo Aircraft Type_Combi         31
Cargo Aircraft Type_Freighter     31
Cargo Aircraft Type_Passenger     31
Cargo Metric TONS                 31
Length: 110, dtype: int64

In [51]:
combined_df.dropna(inplace=True)

In [52]:
combined_df.isna().sum()

Activity Period                   0
Operating Airline_ABX Air         0
Operating Airline_ATA Airlines    0
Operating Airline_Aer Lingus      0
Operating Airline_Aeromexico      0
                                 ..
Cargo Type Code_Mail              0
Cargo Aircraft Type_Combi         0
Cargo Aircraft Type_Freighter     0
Cargo Aircraft Type_Passenger     0
Cargo Metric TONS                 0
Length: 110, dtype: int64

In [53]:
combined_df_test.isna().sum()

Activity Period                   0
Operating Airline_ABX Air         0
Operating Airline_ATA Airlines    0
Operating Airline_Aer Lingus      0
Operating Airline_Aeromexico      0
                                 ..
Cargo Type Code_Express           0
Cargo Type Code_Mail              0
Cargo Aircraft Type_Combi         0
Cargo Aircraft Type_Freighter     0
Cargo Aircraft Type_Passenger     0
Length: 109, dtype: int64

## Above:
    - We have an updated dataframe that contains 1 hot encoded values

## We now need to split Activity Period Column
    - We need to feature engineer this column because it represents month and year
    - We need to split it into two columns so we can account for different seasons ie summer and winter months having different trends
    - This means we have to treat the new month column in a cyclical way

In [54]:
import numpy as np

# Split my activity period to month and year to see better trends
combined_df['Year'] = combined_df['Activity Period'].astype(str).str[:4]
combined_df['Month'] = combined_df['Activity Period'].astype(str).str[4:]

# Convert cols to integer
combined_df['Month'] = combined_df['Month'].astype(float).round().astype(int)
combined_df['Year'] = combined_df['Year'].astype(int)

# Make them cyclical
combined_df['Month_sin'] = np.sin(2 * np.pi * combined_df['Month'] / 12)
combined_df['Month_cos'] = np.cos(2 * np.pi * combined_df['Month'] / 12)
combined_df.drop('Activity Period', axis=1, inplace=True)
combined_df.drop('Month', axis=1, inplace=True)
combined_df

,Operating Airline_ABX Air,Operating Airline_ATA Airlines,Operating Airline_Aer Lingus,Operating Airline_Aeromexico,Operating Airline_Air Berlin,Operating Airline_Air Canada,Operating Airline_Air Canada Jazz,Operating Airline_Air Cargo Carriers,Operating Airline_Air China,Operating Airline_Air France,...,Cargo Type Code_Cargo,Cargo Type Code_Express,Cargo Type Code_Mail,Cargo Aircraft Type_Combi,Cargo Aircraft Type_Freighter,Cargo Aircraft Type_Passenger,Cargo Metric TONS,Year,Month_sin,Month_cos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,8.871509,2016,5.000000e-01,8.660254e-01
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,53.946194,2008,-5.000000e-01,8.660254e-01
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.043999,2007,-1.000000e+00,-1.836970e-16
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,19.023530,2016,-8.660254e-01,-5.000000e-01
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,6.951420,2017,-8.660254e-01,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,39.472272,2013,-2.449294e-16,1.000000e+00
27195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,14.824102,2009,-1.000000e+00,-1.836970e-16
27196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,231.507461,2014,-5.000000e-01,-8.660254e-01
27197,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,99.517572,2017,-1.000000e+00,-1.836970e-16


## do the same for test data

In [55]:
import numpy as np

# Split my activity period to month and year to see better trends
combined_df_test['Year'] = combined_df_test['Activity Period'].astype(str).str[:4]
combined_df_test['Month'] = combined_df_test['Activity Period'].astype(str).str[4:]

# # Convert 'Month' to integer
combined_df_test['Month'] = combined_df_test['Month'].astype(float).round().astype(int)
combined_df_test['Year'] = combined_df_test['Year'].astype(int)


combined_df_test['Month_sin'] = np.sin(2 * np.pi * combined_df_test['Month'] / 12)
combined_df_test['Month_cos'] = np.cos(2 * np.pi * combined_df_test['Month'] / 12)
combined_df_test.drop('Activity Period', axis=1, inplace=True)
combined_df_test.drop('Month', axis=1, inplace=True)
combined_df_test

,Operating Airline_ABX Air,Operating Airline_ATA Airlines,Operating Airline_Aer Lingus,Operating Airline_Aeromexico,Operating Airline_Air Berlin,Operating Airline_Air Canada,Operating Airline_Air Canada Jazz,Operating Airline_Air Cargo Carriers,Operating Airline_Air China,Operating Airline_Air France,...,Activity Type Code_Enplaned,Cargo Type Code_Cargo,Cargo Type Code_Express,Cargo Type Code_Mail,Cargo Aircraft Type_Combi,Cargo Aircraft Type_Freighter,Cargo Aircraft Type_Passenger,Year,Month_sin,Month_cos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2009,-5.000000e-01,-8.660254e-01
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2020,5.000000e-01,8.660254e-01
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2011,-5.000000e-01,-8.660254e-01
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2018,-1.000000e+00,-1.836970e-16
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,2008,-5.000000e-01,8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2016,1.000000e+00,6.123234e-17
6803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2019,-8.660254e-01,5.000000e-01
6804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2015,1.000000e+00,6.123234e-17
6805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,2010,-1.000000e+00,-1.836970e-16


# Need columns to match in both train and test datasets
    - Let's identify what columns are unique in both, we will need to remove them
    - We have some unique columns because our training set and test set have different data

In [442]:
# Make the features the same
# Get the list of features (columns) in each DataFrame
features_train = set(combined_df.columns)
features_test = set(combined_df_test.columns)

# Compare the features between the two DataFrames
unique_features_train = features_train.difference(features_test)
unique_features_test = features_test.difference(features_train)

# Print the results
print("\nFeatures Unique to train:")
print(unique_features_train)
print("\nFeatures Unique to test:")
print(unique_features_test)


Features Unique to train:
{'Cargo Metric TONS'}

Features Unique to test:
set()


# Above:
    - As you can see, the columns are now matched well
    - The target column being unique to the training set is expected

## Above:
    - We now have the same feature/columns for the train and test data (except test data doesnt contain target feature)

# Find out which algorithm is best
    - We are going to use MSE to compare our models
    - Need to find baseline MSE 
    - Going to try different regressor algos and plot them

# Baseline model for MSE to compare to other algorithms

In [56]:
# GET A BASELINE MODEL FOR MSE
from sklearn.metrics import mean_squared_error
import math

# Calculate the mean of the target column (constant prediction)
baseline_prediction = combined_df['Cargo Metric TONS'].mean()

# Create an array of constant predictions
baseline_predictions = [baseline_prediction] * len(combined_df)

# Calculate the MSE for the baseline model
baseline_mse = mean_squared_error(combined_df['Cargo Metric TONS'], baseline_predictions)

print("Baseline MSE: ", baseline_mse)
print("Baseline RMSE: ", math.sqrt(baseline_mse))

Baseline MSE:  187536.0755384708
Baseline RMSE:  433.05435633240177


# We will now test different regression algos and compare it to our baseline MSE and RMSE. If the algo that we are testing has a lower MSE/RMSE than our baseline metrics, then we are in good shape. We will choose the algo with MSE/RMSE closest to 0

# Linear Regression
    - Good first regression model to start off with
    - Continous and supervised

In [58]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import math

Y = combined_df['Cargo Metric TONS']
X = combined_df.drop(columns=['Cargo Metric TONS'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

linear_mse = mean_squared_error(Y_test, Y_pred)

print("Linear MSE: ", linear_mse)
print("Linear RMSE: ", math.sqrt(linear_mse))

Linear MSE:  188062.68993836164
Linear RMSE:  433.66195352873837


# Random Forest

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math

# Assuming you have a DataFrame 'combined_df' with your data
X = combined_df.drop(columns=['Cargo Metric TONS'])  # Features
y = combined_df['Cargo Metric TONS']  # Target variable

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of cross-validation folds
folds = 5

# Define a dictionary of hyperparameters to search
params = {
    'max_depth': [4, 5, 6, 7, 8],
    'n_estimators': [30, 40, 50, 60, 70]
}

# Create a GridSearchCV object for hyperparameter tuning
rf = RandomForestRegressor()
clf_rf = GridSearchCV(rf, params, cv=folds, scoring='neg_mean_squared_error')

# Fit the model with training data
clf_rf.fit(X_train, y_train)

# Get the best hyperparameters found by GridSearchCV
best_hyperparameters = clf_rf.best_params_

# Create a Random Forest Regressor with the best hyperparameters
best_rf = RandomForestRegressor(max_depth=best_hyperparameters['max_depth'], n_estimators=best_hyperparameters['n_estimators'])

# Fit the model with training data using the best hyperparameters
best_rf.fit(X_train, y_train)

# Make predictions on the training data
y_pred = best_rf.predict(X_test)

# Calculate the training set MSE
rf_mse_train = mean_squared_error(y_test, y_pred)

In [452]:
print("Random Forest MSE:", rf_mse_train)
print("Random Forest RMSE:", math.sqrt(rf_mse_train))

Random Forest MSE: 187774.77476320713
Random Forest RMSE: 433.3298683026675


# LASSO
    - It is good at improving model accuracy through regularization/penalization

In [61]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


# Define the number of cross-validation folds
folds = 5

# Define a dictionary of hyperparameters to search
params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],  # L1 regularization strength
    'tol': [1e-4, 1e-5, 1e-6],  # Tolerance for stopping criteria
    'max_iter': [10000, 20000, 30000]  # Maximum number of iterations
}

# Create a Lasso Regressor
lasso = Lasso(max_iter=20000)  # Increase max_iter


# Create a GridSearchCV object for hyperparameter tuning
clf_lasso = GridSearchCV(lasso, params, cv=folds, scoring='neg_mean_squared_error')

# Define training data
X_train = combined_df.drop(columns=['Cargo Metric TONS'])
y_train = combined_df['Cargo Metric TONS']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model with training data
clf_lasso.fit(X_train, y_train)

# Get the best hyperparameters found by GridSearchCV
best_hyperparameters = clf_lasso.best_params_

# Create a Lasso Regressor with the best hyperparameters
best_lasso = Lasso(alpha=best_hyperparameters['alpha'])

# Fit the model with training data using the best hyperparameters
best_lasso.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_lasso.predict(X_test)

In [62]:
lasso_mse_train = mean_squared_error(y_test, y_pred)
print("Lasso MSE:", lasso_mse_train)
print("Lasso RMSE:", math.sqrt(lasso_mse_train))

Lasso MSE: 187115.343367179
Lasso RMSE: 432.56831063680454


# kNN
    - Algo that isn't sensitive to outliers
    - May produce better MSE/RMSE

In [462]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV


# Define the number of cross-validation folds
folds = 5

# Define a dictionary of hyperparameters to search
params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Create a K-Nearest Neighbors Regressor
knn = KNeighborsRegressor()

# Create a GridSearchCV object for hyperparameter tuning
clf_knn = GridSearchCV(knn, params, cv=folds, scoring='neg_mean_squared_error')

# Define training data
X_train = combined_df.drop(columns=['Cargo Metric TONS'])
y_train = combined_df['Cargo Metric TONS']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model with training data
clf_knn.fit(X_train, y_train)

# Get the best hyperparameters found by GridSearchCV
best_hyperparameters = clf_knn.best_params_

# Create a K-Nearest Neighbors Regressor with the best hyperparameters
best_knn = KNeighborsRegressor(n_neighbors=best_hyperparameters['n_neighbors'], 
                              weights=best_hyperparameters['weights'], 
                              p=best_hyperparameters['p'])

# Fit the model with training data using the best hyperparameters
best_knn.fit(X_train, y_train)

# Make predictions on the training data
y_pred = best_knn.predict(X_test)

# calculate mse
knn_mse_train = mean_squared_error(y_test, y_pred)

In [463]:
print("kNN MSE:", knn_mse_train)
print("kNN RMSE:", math.sqrt(knn_mse_train))

kNN MSE: 208336.6374022721
kNN RMSE: 456.43908399946656


# XGBoost

In [63]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import math

# Assuming you already have your dataframe 'combined_df'

X = combined_df.drop(columns=['Cargo Metric TONS'])
y = combined_df['Cargo Metric TONS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost regressor
xgb_model = XGBRegressor()

# Define a parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Create a new XGBoost regressor with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)

# Train the model with the best hyperparameters
best_xgb_model.fit(X_train, y_train)

# Make predictions
predictions = best_xgb_model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, predictions)
print("MSE for XGBoost is: ", mse)
print("RMSE for XGBoost is: ", math.sqrt(mse))

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50}
MSE for XGBoost is:  186948.49567618137
RMSE for XGBoost is:  432.375410582264


# Ridge
    - Similar to Lasso
    - It is good at improving model accuracy through regularization/penalization

In [65]:
from sklearn.linear_model import Ridge

# Define the number of cross-validation folds
folds = 5

# Define a dictionary of hyperparameters to search
params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]  # L2 regularization strength
}

# Create a Ridge Regressor
ridge = Ridge()

# Create a GridSearchCV object for hyperparameter tuning
clf_ridge = GridSearchCV(ridge, params, cv=folds, scoring='neg_mean_squared_error')

# Define training data
X = combined_df.drop(columns=['Cargo Metric TONS'])
y = combined_df['Cargo Metric TONS']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model with training data
clf_ridge.fit(X_train, y_train)

# Get the best hyperparameters found by GridSearchCV
best_hyperparameters = clf_ridge.best_params_
best_alpha = best_hyperparameters['alpha']  # Extract the best alpha value

# Create a Ridge Regressor with the best hyperparameters
best_ridge = Ridge(alpha=best_alpha)

# Fit the model with training data using the best hyperparameters
best_ridge.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_ridge.predict(X_test)

In [66]:
ridge_mse_train = mean_squared_error(y_test, y_pred)
print("Ridge MSE:", ridge_mse_train)
print("Ridge RMSE:", math.sqrt(ridge_mse_train))

Ridge MSE: 186945.56887628883
Ridge RMSE: 432.3720260103431


# OVERALL: Ridge has the slightest advantage as most accurate model, but very slightly
    - Lasso RMSE: 432.568
    - XGBoost RMSE:  432.375
    - Ridge RMSE: 432.372
    - Out of all of the models, I would choose Ridge as being the most accurate as well as quickest to run giving us the best performance overall
    - But if you prefer to use Lasso or XGBoost, I do not see any problem using either of those models

In [68]:
# Predicted target values for test set
# cleaned test dataset from earlier
X = combined_df_test

# where I save target column values
y_target_col= best_ridge.predict(X)

# create df
y_target_col_df = pd.DataFrame({'Predicted Cargo Metric TONS': y_target_col})

y_target_col_df

# set csv file
y_target_col_df.to_csv('data_challene_1_predictions_ridge.csv')